In [31]:
import pandas as pd
import os

In [32]:
# read in original

original = pd.read_csv('./../../FreeText-Dataset-31-USERS.csv')

In [33]:
DIR = "./chromium_isolated"

dirs = [
    DIR + "_freetext_1-7/",
    DIR + "_freetext_8-14/",
    DIR + "_freetext_15-21/",
    DIR + "_freetext_22-28/",
    DIR + "_freetext_29-31/",
]

In [34]:
# read in timings
timings_list = []

for dir in dirs[:-1]:
    for set in range(0, 105):
        path = os.path.join(dir, f'./text-timing-data({set}).csv')
        df = pd.read_csv(path)
        timings_list.append(df)

for set in range(0, 45):
    path = os.path.join(dirs[-1], f'./text-timing-data({set}).csv')
    df = pd.read_csv(path)
    timings_list.append(df)


assert len(timings_list) == 31 * 15


In [35]:
# read in keys
users = range(1, 8)
keys_list = []


for (set, dir) in enumerate(dirs[:-1]):
    for user in users:
        for set_i in range(1, 16):
            path = os.path.join(
                dir, f'free-text-output/user{user+( set *7)}-{set_i}.csv')
            df = pd.read_csv(path)
            keys_list.append(df)

for user in range(29, 32):
    for set_i in range(1, 16):
        path = os.path.join(
            dirs[-1], f'free-text-output/user{user}-{set_i}.csv')
        df = pd.read_csv(path)
        keys_list.append(df)


assert len(keys_list) == len(timings_list) == 31 * 15


In [36]:
# combine keys and timings

dataframes = []
for user in range(1, 32):
    user_dataframes = []
    for set in range(1, 16):
        idx = ((user - 1) * 15) + (set - 1)
        last = idx
        timings = timings_list[idx]
        keys = keys_list[idx]

        # trim warmups
        timings = timings.iloc[8:]

        # get orignal data for user and set
        originals = original.loc[(original['user'] == user) & (original['set'] == set)]

        assert len(timings) == len(keys) == len(originals)


        df = pd.DataFrame({
            'user': user,
            'set': set,
            'timestamp': timings['timestamp'].to_list(),
            'key': keys['key'].to_list(),
            'pressed_key': timings['key'].to_list()
        })

        assert len(df) == len(originals)
        assert df['key'].to_list() == originals['key'].to_list()

        user_dataframes.append(df)


    assert len(user_dataframes) == 15
    user_dataframe = pd.concat(user_dataframes)

    dataframes.append(user_dataframe)


In [37]:
# combine into one dataset

complete = pd.concat(dataframes)

assert len(complete) == len(original)

complete.to_csv('./complete.csv', index=False)